The code calculates various date and time related features based on eva.ru forum posts publication dates.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
Data='/home/kate/Projects/eva/Data/Main'

ts_extension_filename='TimeTrends_ext.csv'
ts_extension_full_filename=os.path.join(Data, ts_extension_filename)

In [3]:
ts_ext = pd.read_csv(ts_extension_full_filename, error_bad_lines=False, index_col=False) 

In [4]:
len(ts_ext)

9455613

In [5]:
ts_ext.columns

Index(['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'cnt_childs', 'cnt_immediate_childs'],
      dtype='object')

In [6]:
ts_ext.dtypes

Message_Id               int64
Timestamp               object
Topic_1st_Message       object
Parent_Id                int64
Author_Id                int64
author                  object
Topic_Id                 int64
Topic                   object
Chapter_Id               int64
Chapter                 object
cnt_childs               int64
cnt_immediate_childs     int64
dtype: object

In [7]:
ts_ext['Timestamp']=pd.to_datetime(ts_ext['Timestamp'], format='%Y-%m-%d %H:%M:%S')

In [8]:
ts_ext.head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,author,Topic_Id,Topic,Chapter_Id,Chapter,cnt_childs,cnt_immediate_childs
0,39333688,2008-09-01 12:06:00,Y,0,186982,Анулиса,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,6,2
1,39333763,2008-09-01 12:09:00,N,0,186982,Анулиса,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,1,1
2,39357288,2008-09-02 09:26:00,N,0,130919,CлонёнОК,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,2,2
3,39383519,2008-09-03 07:54:00,N,0,130919,CлонёнОК,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,0,0
4,39410471,2008-09-04 07:08:00,N,0,130919,CлонёнОК,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,0,0


In [9]:
ts_ext['year']=ts_ext['Timestamp'].dt.year
ts_ext['month']=ts_ext['Timestamp'].dt.month
ts_ext['month_name']=ts_ext['Timestamp'].dt.month_name()
ts_ext['day_of_week']=ts_ext['Timestamp'].dt.day_of_week #Monday : day_of_week = 0
ts_ext['day_of_week_name']=ts_ext['Timestamp'].dt.day_name()
ts_ext['hour'] = ts_ext['Timestamp'].dt.hour

In [10]:
def day_part(hour):
    if hour in [4,5]:
        return 'dawn'
    elif hour in [6,7]:
        return 'early morning'
    elif hour in [8,9,10]:
        return 'late morning'
    elif hour in [11,12,13]:
        return 'noon'
    elif hour in [14,15,16]:
        return 'afternoon'
    elif hour in [17, 18,19]:
        return 'evening'
    elif hour in [20, 21, 22]:
        return 'night'
    elif hour in [23,24,1,2,3]:
        return 'midnight'

In [11]:
ts_ext['day_part'] = ts_ext['hour'].apply(day_part)

In [12]:
ts_ext['is_weekend'] = np.where(ts_ext['day_of_week'].isin([5,6]), 1,0)

In [13]:
len(ts_ext)

9455613

## National Holidays, Observance, other not working days, working days replacments
based on https://www.timeanddate.com/

In [14]:
First_Year=ts_ext['year'].min()
Last_Year=ts_ext['year'].max()

In [15]:
countries=['russia']

In [16]:
from bs4 import BeautifulSoup
import urllib.request

In [17]:
Holiday_df=pd.DataFrame()
for country in countries:
    print(country)
    for year in range(First_Year,Last_Year+1):
        print(year)
        html_page = urllib.request.urlopen('https://www.timeanddate.com/holidays/%s/%s'%(country,year))
        soup = BeautifulSoup(html_page)
        table = soup.find_all('table', {'id': 'holidays-table'})
        df = pd.read_html(str(table))[0]
        df.columns = df.columns.droplevel(0)
        if country=='russia':
            df['Details']=''
        df.columns=['Date','Dummy','Name','Type','Details']
        df=df[['Date','Name','Type','Details']]
        df.dropna(inplace=True)
        df['Date']=str(year)+' '+df['Date']
        df['Date']=pd.to_datetime(df['Date'], format='%Y %b %d')
        df['country']=country
        Holiday_df=Holiday_df.append(df)

russia
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [18]:
len(Holiday_df)

562

In [19]:
Holiday_df['Type'].unique()

array(['National holiday', 'National holiday, Orthodox', 'Observance',
       'Muslim', 'Season', 'Observance, Orthodox',
       'Working day (replacement)', 'Paid leave', 'Substitute Holiday'],
      dtype=object)

* I am interested only in 'National holiday', 'National holiday, Orthodox', 'Observance','Observance, Orthodox', 'Working day (replacement)', 'Paid leave', 'Substitute Holiday' holidays

In [20]:
Holiday_df=Holiday_df[Holiday_df['Type'].isin(['National holiday', 'National holiday, Orthodox', 'Observance','Observance, Orthodox','Working day (replacement)', 'Paid leave', 'Substitute Holiday'])]

In [21]:
len(Holiday_df)

395

In [22]:
Holiday_df.head()

,Date,Name,Type,Details,country
0,2007-01-01,New Year's Day,National holiday,,russia
1,2007-01-02,New Year Holiday,National holiday,,russia
2,2007-01-03,New Year Holiday,National holiday,,russia
3,2007-01-04,New Year Holiday,National holiday,,russia
4,2007-01-05,New Year Holiday,National holiday,,russia


### Duplicates in Holidays

In [23]:
Holiday_df.groupby(['Date']).size().reset_index(name='counts').sort_values('counts', ascending=False)

,Date,counts
224,2016-05-01,2
150,2013-05-10,2
335,2020-04-19,2
0,2007-01-01,1
258,2018-01-07,1
...,...,...
128,2012-06-09,1
127,2012-05-09,1
126,2012-05-01,1
125,2012-04-30,1


In [24]:
Holiday_df[Holiday_df['Date'].isin(['2016-05-01','2013-05-10','2020-04-19'])]

,Date,Name,Type,Details,country
21,2013-05-10,Defender of the Fatherland Day holiday,National holiday,,russia
22,2013-05-10,Victory Day Holiday,National holiday,,russia
17,2016-05-01,Orthodox Easter Day,"Observance, Orthodox",,russia
18,2016-05-01,Spring and Labor Day,National holiday,,russia
40,2020-04-19,Orthodox Easter Day,"Observance, Orthodox",,russia
41,2020-04-19,Non-Working Day (COVID-19),Paid leave,,russia


* removing non-holidays

In [25]:
Holiday_df=Holiday_df[(~((Holiday_df['Date'].isin(['2013-05-10','2016-05-01','2020-04-19'])) & (Holiday_df['Name'].isin(['Defender of the Fatherland Day holiday','Orthodox Easter Day']))) )]

In [26]:
Holiday_df.groupby(['Date']).size().reset_index(name='counts').sort_values('counts', ascending=False)

,Date,counts
0,2007-01-01,1
269,2018-05-01,1
267,2018-04-28,1
266,2018-04-08,1
265,2018-03-09,1
...,...,...
128,2012-06-09,1
127,2012-05-09,1
126,2012-05-01,1
125,2012-04-30,1


In [27]:
len(Holiday_df)

392

In [28]:
ts_ext['Date']=ts_ext['Timestamp'].astype(str)
ts_ext['Date']=ts_ext['Date'].str[0:10]
ts_ext['Date']=pd.to_datetime(ts_ext['Date'], format='%Y-%m-%d')

In [29]:
ts_ext=pd.merge(ts_ext, Holiday_df, left_on='Date', right_on='Date', how='left')

In [30]:
len(ts_ext)

9455613

In [31]:
ts_ext['Type']=ts_ext['Type'].fillna('None')
ts_ext['Type']=ts_ext['Type'].astype(str)

ts_ext['Name']=ts_ext['Name'].fillna('None')
ts_ext['Name']=ts_ext['Name'].astype(str)

In [32]:
ts_ext['is_ruNationalHoliday']=[1 if x.find('National holiday')!=-1 else 0 for x in ts_ext['Type']]

In [33]:
ts_ext['is_ruObservance']=[1 if x.find('Observance')!=-1 else 0 for x in ts_ext['Type']]

In [34]:
ts_ext['is_ruOtherNotWorkingDay']=[1 if ((x.find('Paid leave')!=-1) | (x.find('Substitute Holiday')!=-1)) else 0 for x in ts_ext['Type']]

In [35]:
ts_ext['is_weekend'] = [2 if x.find('Working day (replacement)')!=-1 else y for (x,y) in zip(ts_ext['Type'],ts_ext['is_weekend'])]

In [36]:
ts_ext.columns

Index(['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'cnt_childs', 'cnt_immediate_childs', 'year', 'month', 'month_name',
       'day_of_week', 'day_of_week_name', 'hour', 'day_part', 'is_weekend',
       'Date', 'Name', 'Type', 'Details', 'country', 'is_ruNationalHoliday',
       'is_ruObservance', 'is_ruOtherNotWorkingDay'],
      dtype='object')

In [37]:
ts_ext=ts_ext[['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'cnt_childs', 'cnt_immediate_childs', 'Date', 'year', 'month', 'day_of_week',
       'hour', 'day_part', 'is_weekend',  'is_ruNationalHoliday', 'is_ruObservance',
       'is_ruOtherNotWorkingDay',  'Name', 'Type']]

In [38]:
ts_ext.columns=['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'cnt_childs', 'cnt_immediate_childs', 'Date', 'year', 'month', 'day_of_week',
       'hour', 'day_part', 'is_weekend',  'is_ruNationalHoliday', 'is_ruObservance',
       'is_ruOtherNotWorkingDay',  'HolidayName', 'HolidayType']

In [39]:
ts_ext[ts_ext['is_weekend']==2].head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,author,Topic_Id,Topic,Chapter_Id,Chapter,...,month,day_of_week,hour,day_part,is_weekend,is_ruNationalHoliday,is_ruObservance,is_ruOtherNotWorkingDay,HolidayName,HolidayType
117,41045627,2008-11-01 13:09:00,N,0,169848,Тигрица!,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,5,13,noon,2,0,0,0,Working day - Unity Day,Working day (replacement)
118,41052824,2008-11-01 17:05:00,N,41045627,100060,Ануля,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,5,17,evening,2,0,0,0,Working day - Unity Day,Working day (replacement)
119,41057209,2008-11-01 20:15:00,N,0,61428,ЭрЛи,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,5,20,night,2,0,0,0,Working day - Unity Day,Working day (replacement)
120,41059143,2008-11-01 21:25:00,N,41057209,163094,Аануля,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,5,21,night,2,0,0,0,Working day - Unity Day,Working day (replacement)
264220,74733911,2012-04-28 11:28:00,N,0,0,Anonymous,2853583,Что читаете 5-летке?,137,Детская психология и развитие,...,4,5,11,noon,2,0,0,0,Special working day,Working day (replacement)


In [40]:
ts_ext[ts_ext['is_ruNationalHoliday']==1].head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,author,Topic_Id,Topic,Chapter_Id,Chapter,...,month,day_of_week,hour,day_part,is_weekend,is_ruNationalHoliday,is_ruObservance,is_ruOtherNotWorkingDay,HolidayName,HolidayType
131,41111078,2008-11-04 12:37:00,N,41102397,130919,CлонёнОК,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,1,12,noon,0,1,0,0,Unity Day,National holiday
132,41124389,2008-11-04 22:05:00,N,41057209,49712,Л я л ь к а,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,1,22,night,0,1,0,0,Unity Day,National holiday
245,42933847,2009-01-05 19:23:00,N,0,79787,Йодо,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,1,0,19,evening,0,1,0,0,New Year Holiday,National holiday
246,42935441,2009-01-05 20:39:00,N,42933847,130919,CлонёнОК,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,1,0,20,night,0,1,0,0,New Year Holiday,National holiday
247,42940318,2009-01-05 23:28:00,N,42935441,79787,Йодо,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,1,0,23,midnight,0,1,0,0,New Year Holiday,National holiday


In [41]:
ts_ext[ts_ext['is_ruObservance']==1].head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,author,Topic_Id,Topic,Chapter_Id,Chapter,...,month,day_of_week,hour,day_part,is_weekend,is_ruNationalHoliday,is_ruObservance,is_ruOtherNotWorkingDay,HolidayName,HolidayType
0,39333688,2008-09-01 12:06:00,Y,0,186982,Анулиса,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,9,0,12,noon,0,0,1,0,Day of Knowledge,Observance
1,39333763,2008-09-01 12:09:00,N,0,186982,Анулиса,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,9,0,12,noon,0,0,1,0,Day of Knowledge,Observance
125,41085885,2008-11-03 05:49:00,N,41079755,130919,CлонёнОК,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,0,5,dawn,0,0,1,0,Unity Day Weekend,Observance
126,41085899,2008-11-03 05:58:00,N,41068879,130919,CлонёнОК,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,0,5,dawn,0,0,1,0,Unity Day Weekend,Observance
127,41088111,2008-11-03 10:48:00,N,41085885,61428,ЭрЛи,1441215,Оч. умелые мамашки-3,137,Детская психология и развитие,...,11,0,10,late morning,0,0,1,0,Unity Day Weekend,Observance


In [42]:
ts_ext[ts_ext['is_ruOtherNotWorkingDay']==1].head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,author,Topic_Id,Topic,Chapter_Id,Chapter,...,month,day_of_week,hour,day_part,is_weekend,is_ruNationalHoliday,is_ruObservance,is_ruOtherNotWorkingDay,HolidayName,HolidayType
435992,100483160,2020-04-10 13:16:00,N,82735890,737936,id557093451 +,3184984,"Не знаю, чем занять 9-летнего ребенка",137,Детская психология и развитие,...,4,4,13,noon,0,0,0,1,Non-Working Day (COVID-19),Paid leave
632987,100394734,2020-03-28 14:39:00,N,92504867,737213,fb2979030935482064 +,3425518,положительные истории про детей после тяжелой ...,137,Детская психология и развитие,...,3,5,14,afternoon,1,0,0,1,Non-Working Day (COVID-19),Paid leave
632988,100394794,2020-03-28 14:51:00,N,100394734,0,Anonymous,3425518,положительные истории про детей после тяжелой ...,137,Детская психология и развитие,...,3,5,14,afternoon,1,0,0,1,Non-Working Day (COVID-19),Paid leave
632989,100395025,2020-03-28 15:35:00,N,100394734,737218,fb2832938803461352 +,3425518,положительные истории про детей после тяжелой ...,137,Детская психология и развитие,...,3,5,15,afternoon,1,0,0,1,Non-Working Day (COVID-19),Paid leave
632990,100395862,2020-03-28 18:27:00,N,92361591,0,Anonymous,3425518,положительные истории про детей после тяжелой ...,137,Детская психология и развитие,...,3,5,18,evening,1,0,0,1,Non-Working Day (COVID-19),Paid leave


In [43]:
ts_ext.to_csv(ts_extension_full_filename, header=True, index=False)

In [44]:
len(ts_ext)

9455613